In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import threading
import pygame
import time

# Константы
VIDEO_PATH = "video\people.mp4"  # Путь к видеофайлу
ALARM_SOUND = "alarm.mp3"  # Файл звукового сигнала
REAL_DISTANCE_THRESHOLD = 45  # Порог для близости людей (в сантиметрах)
CONFIDENCE_THRESHOLD = 0.4  # Порог уверенности для обнаружения
AVERAGE_PERSON_HEIGHT = 170  # Средний рост человека в сантиметрах
CUDA_OR_CPU = "_"
FRAME_PAUSE = 0.01

# Инициализация pygame для звукового сигнала
def initialize_audio():
    pygame.mixer.init()
    pygame.mixer.music.load(ALARM_SOUND)

# Воспроизведение звука тревоги
def play_alarm():
    if not pygame.mixer.music.get_busy():
        pygame.mixer.music.play()

# Загрузка модели
def load_model():
    if CUDA_OR_CPU == "cuda":
        model = YOLO('yolov8x.pt')
        model.to('cuda')
    else:
        model = YOLO('yolov8n.pt')
        model.to('cpu')
    return model

# Обработка кадра для обнаружения людей и проверки близости
def process_frame(frame, model, class_index):
    results = model.predict(source=frame, conf=CONFIDENCE_THRESHOLD)
    boxes, heights = extract_person_data(results, class_index)
    close_pairs = find_close_pairs(boxes, heights)
    return boxes, close_pairs

# Извлечение данных о людях из результатов модели
def extract_person_data(results, class_index):
    boxes = []
    heights = []
    for box in results[0].boxes:
        if int(box.cls) == class_index:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            person_height_pixels = y2 - y1
            boxes.append((x1, y1, x2, y2))
            heights.append(person_height_pixels)
    return boxes, heights

# Поиск пар близких людей с учетом реального расстояния и размера bbox
def find_close_pairs(boxes, heights):
    close_pairs = set()
    for i, ((x1_a, y1_a, x2_a, y2_a), height_a) in enumerate(zip(boxes, heights)):
        scale_a = AVERAGE_PERSON_HEIGHT / height_a  # См/пиксель для человека i
        for j, ((x1_b, y1_b, x2_b, y2_b), height_b) in enumerate(zip(boxes, heights)):
            if i != j:
                scale_b = AVERAGE_PERSON_HEIGHT / height_b  # См/пиксель для человека j
                avg_scale = (scale_a + scale_b) / 2

                # Проверка разницы в размерах bbox
                real_height_a = height_a * avg_scale
                real_height_b = height_b * avg_scale
                if abs(real_height_a - real_height_b) > REAL_DISTANCE_THRESHOLD:
                    continue

                # Вычисление минимального расстояния между границами
                x_distance = max(0, max(x1_a, x1_b) - min(x2_a, x2_b))
                y_distance = max(0, max(y1_a, y1_b) - min(y2_a, y2_b))
                pixel_distance = np.sqrt(x_distance ** 2 + y_distance ** 2)
                real_distance = pixel_distance * avg_scale  # Перевод в сантиметры

                if real_distance < REAL_DISTANCE_THRESHOLD:
                    close_pairs.add(i)
                    close_pairs.add(j)
    return close_pairs

# Рисование рамок и добавление меток на кадр
def draw_boxes(frame, boxes, close_pairs):
    alarm_triggered = False
    for i, (x1, y1, x2, y2) in enumerate(boxes):
        color = (0, 0, 255) if i in close_pairs else (255, 0, 0)
        if i in close_pairs:
            alarm_triggered = True
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        label = "person"
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return alarm_triggered

# Основной цикл обработки видео
def process_video(video_path, model, class_index):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Не удалось открыть видеофайл.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Конец видео.")
            break

        boxes, close_pairs = process_frame(frame, model, class_index)
        alarm_triggered = draw_boxes(frame, boxes, close_pairs)

        if alarm_triggered:
            threading.Thread(target=play_alarm, daemon=True).start()

        cv2.imshow("Recorded Video Person Detection", frame)

        # Добавляем паузу между кадрами (например, 0.1 секунды)
        time.sleep(FRAME_PAUSE)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Главная функция
if __name__ == "__main__":
    initialize_audio()
    model = load_model()
    class_index = list(model.names.values()).index('person')
    process_video(VIDEO_PATH, model, class_index)
    pygame.mixer.quit()
